In [1]:
%run 0.0_Dependance.ipynb

In [2]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])



target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)

#Standardisation
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_v2_top_1000 = load_create_dftop("df_score_random_v1.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_v2_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_v2_top_1000['Test_log_loss'] = log_losses
df_score_random_v2_top_1000 = df_score_random_v2_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v2_top_1000,"df_score_random_v2_top_1000.joblib")

['df_score_random_v2_top_1000.joblib']

In [4]:
df_score_random_v2_top_1000 = load("df_score_random_v2_top_1000.joblib")
df_score_random_v2_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
21038,800,6,0.06,0.5,1,0.2,8,1,0,multi:softprob,mlogloss,gbtree,-0.429331,0.013338,0.439469
10912,600,6,0.05,0.7,1,0.2,6,0.3,0,multi:softprob,mlogloss,gbtree,-0.428540,0.011796,0.439571
30615,400,5,0.06,0.5,1,0.2,5,0.9,2,multi:softprob,mlogloss,gbtree,-0.430328,0.012239,0.439572
7623,500,6,0.05,0.5,1,0.2,8,0.2,8,multi:softprob,mlogloss,gbtree,-0.430450,0.012727,0.439592
28700,700,6,0.05,0.5,1,0.2,5,0.4,6,multi:softprob,mlogloss,gbtree,-0.429454,0.012705,0.439632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31075,700,6,0.09,0.7,0.75,0.2,6,0.1,3,multi:softprob,mlogloss,gbtree,-0.430288,0.014946,0.452621
30654,900,6,0.1,0.7,0.75,0.2,5,0.7,2,multi:softprob,mlogloss,gbtree,-0.430161,0.013673,0.452664
31215,700,6,0.09,0.7,0.75,0.2,6,0.3,2,multi:softprob,mlogloss,gbtree,-0.430228,0.013956,0.452979
2201,800,5,0.1,0.7,0.75,0.2,5,0.8,3,multi:softprob,mlogloss,gbtree,-0.430540,0.013308,0.453784
